In [1]:
import sys
!{sys.executable} -m pip install qdrant-client ollama sqlalchemy


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [17]:
import os, json
import pandas as pd
from modules import qdrant
from modules.sqlite import write_df_to_sqlite
from modules.archiver import generate_knowledge
from modules.vectorizer import vectorize_dataframe_columns
from modules.archiver import convert_pd_object_values_to_str

file_name = '20251120-mfi-products.csv'
delimiter = ';'
collection_name = 'test_collection'


def read_csv_file(f, d):
    return pd.read_csv(os.path.join(os.getcwd(),f"input/{f}"), delimiter=d)


# Load data, process it, and store in SQLite and Qdrant
def load_data_and_process():
    # Load the CSV file into a DataFrame
    df = read_csv_file(file_name, delimiter)
    relevant_columns = [0,1,5,7,8,9,10,11,12,13,14,16,17,18,19,20,21]
    df = df.iloc[:,relevant_columns]

    # store the dataframe in sqlite
    res = write_df_to_sqlite(df)
    if not res:
        print("Error storing DataFrame in SQLite.")

    # Vectorize the DataFrame
    df_vec = vectorize_dataframe_columns(df) # @to do: check if vector length is covering all relevant columns

    # Store the vectors in Qdrant
    res = qdrant.store_vectors_in_qdrant(df_vec, collection_name=collection_name)
    if not res:
        print("Error storing vectors in Qdrant.")

    return generate_knowledge(df)


knowledge = load_data_and_process()
# print(read_csv_file(file_name, delimiter).info())
print("Completed initialization and data processing.")

Completed initialization and data processing.


In [27]:
import ollama
from modules import input, qdrant, vectorizer

models = ['mistral','stablelm2','avr/sfr-embedding-mistral']
model = models[2] # Choose the model to use


# Generate answer based on prompt
def generate_answer(p, m='mistral'):
    return ollama.generate(
        model=m,
        prompt=p
    )


# Get user input
def get_user_input():
    # Get and validate user input
    qry = input.get_user_input("Enter search term")
    if input.validate_search_term(qry):
        # Proceed with search
        pass
    return qry


# Generate SQL query based on user prompt
def agent_sql_query(q, k, m):
    # Create the agent information prompt
    agent_info = f"You are an AI language model assistant. Your task is to find matching products to the {q} using only results from a product database with this schema: {k}. All data are stored within one table. Create an SQL query that returns the product name aliases and descriptions of the matching products."
    res = generate_answer(agent_info, m)
    return res['response']


# Perform vector search based on user prompt
def agent_vector_search(q, k, m):

    print(f"model: {m}")

    # agent_info = f"You are an AI language model assistant. Your task is to find matching products to the {search_query} using only results from a product database that is described by {knowledge}. Create an embedding vector for the search term and use it to search the vector database for the most relevant products."
    # agent_info = f"Based on the following search result: {search_result[0].payload}, provide a concise answer about the product."
    agent_info = (
        f"You are an AI language model assistant. " +
        f"Your task is to find the top 3 most relevant matching products in a product database that is described by {k}. " +
        f"The search is performed using vector search with cosine similarity. The search term is {q}. " +
        f"Create a prompt for the vector search."
    )

    # print(f"agent_info: {agent_info}")

    res = generate_answer(agent_info, m)
    return res['response']


# Main execution
search_query = get_user_input()

answer = None

answer = agent_sql_query(search_query, knowledge, model)
# answer = agent_vector_search(search_query, knowledge, model=model)
# @to do: log response time for statistical analysis
print(answer)



    SELECT product_name, description, alias
    FROM products p
    JOIN products_aliases pa ON p.id = pa.product_id
    WHERE p.id = 1 AND pa.alias LIKE '%%%s' % s is a wildcard character used to match any product name aliases with the same name.

```sql
product_name, description, alias
---------------------------------------
'AI', "AIs are artificial intelligence algorithms that process and analyze data to perform tasks normally done by humans"
'AI'
'AI', 'AI is an AI that uses machine learning and deep learning to analyze data.'
'AI', 'AI is an AI that uses artificial neural networks to learn from data.'
```

## Answer:

    SELECT product_name, description, alias
    FROM products p
    JOSQL query returns the product name aliases and descriptions of matching products.
    FROM products p
    JOIN products_aliases pa ON p.id = pa.product_id
    WHERE p.id = 1 AND pa.alias LIKE '%%%s' % s is a wild product aliases with the same name.
```sql
product_name, description, alias
-------

In [23]:
prompt = "Perform a vector search on the following dataset to find the most relevant nutrition products. Use cosine similarity as the comparison metric. The search term is 'fat powder for a vegan nutrition product'."
# search_term = ["fat powder", "vegan", "nutrition product"] # Product 132
# search_term = "fat powder for a vegan savoury product" # Product 132
search_term = "vegan savoury product"
# strongly depends on product description details or product group?
search_term = "fat powder"
# strongly depends on product group?
search_term = "fat powder in savoury product segment"
search_term = "casein"

prompt_vector = vectorizer.generate_embeddings(str(search_term))

In [24]:
from modules import qdrant, vectorizer

# Perform search in Qdrant
def search(q, c, l):
    return qdrant.perform_search_in_qdrant(
        query = q,
        collection_name = c,
        limit = l
    )


answer = search(prompt_vector, 'test_collection', 5)
# print(answer)

In [25]:
for i in range(len(answer)):
    print(f"Score: {answer[i].score}: {answer[i].payload}")

Score: 0.7164975720950417: {'product_name': 'Product 156', 'product_description': 'Extruded Calcium Caseinate '}
Score: 0.7156343279096176: {'product_name': 'Product 157', 'product_description': 'Extruded Calcium Caseinate '}
Score: 0.7108672972662002: {'product_name': 'Product 158', 'product_description': 'Extruded Sodium Caseinate'}
Score: 0.7107120124108477: {'product_name': 'Product 159', 'product_description': 'Extruded Sodium Caseinate'}
Score: 0.7079375084517243: {'product_name': 'Product 160', 'product_description': 'Extra fine extruded Sodium Caseinate'}
